In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head(10)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,10556855,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION...,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,10558440,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRI...,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,10566033,ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,10570430,ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,10571689,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


Optimization Attempt: 1

In [2]:
# Drop the non-beneficial columns
clean_application_df = application_df.drop(columns=["EIN", "NAME","USE_CASE", "SPECIAL_CONSIDERATIONS"])
clean_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,1,0,5000,1
1,T3,Independent,C2000,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Trust,1,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
clean_application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
appCounts = clean_application_df["APPLICATION_TYPE"].value_counts()
appCounts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(appCounts[appCounts<500].index)
# Replace in dataframe
for app in application_types_to_replace:
   clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
clean_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at INCOME_AMOUNT value counts for binning
incomeCounts= clean_application_df["INCOME_AMT"].value_counts()
incomeCounts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [7]:
# Choose a cutoff value and create a list of incomes to be replaced
# use the variable name `incomes_to_replace`
incomes_to_replace = list(incomeCounts[incomeCounts<1000].index)

# Replace in dataframe
for inc in incomes_to_replace:
    clean_application_df['INCOME_AMT'] = clean_application_df['INCOME_AMT'].replace(inc,"Other")

# Check to make sure binning was successful
clean_application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classCounts = clean_application_df["CLASSIFICATION"].value_counts()
classCounts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >10
classCounts10 = classCounts[classCounts>10]
classCounts10

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classCounts10[classCounts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
clean_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2143
C3000     1918
C2100     1883
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1256        2
C1246        2
C1267        2
C2190        1
C4200        1
C1370        1
C5200        1
C1820        1
C2600        1
C1900        1
C1248        1
C6100        1
C1236        1
C1732        1
C2570        1
C1580        1
C3700        1
C2500        1
C1570        1
C1283        1
C2380        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: CLASSIFICATION, dtype: int64

In [11]:
new_application_df= pd.get_dummies(clean_application_df)
new_application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_C8210,CLASSIFICATION_Other,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0


In [12]:
# Split the preprocessed data into our features and target arrays (X,y)
y= new_application_df["IS_SUCCESSFUL"]
X= new_application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing datatset
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [13]:
y[:5]

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [14]:
X[:5]

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,CLASSIFICATION_C8210,CLASSIFICATION_Other,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1,108590,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=n_input_features,units=85,activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=20,activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10,activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 85)                6460      
                                                                 
 dense_1 (Dense)             (None, 20)                1720      
                                                                 
 dense_2 (Dense)             (None, 10)                210       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 8,401
Trainable params: 8,401
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [18]:
# Train the model
fit_model= nn.fit(X_train,y_train,epochs=175)

Epoch 1/175
804/804 [==============================] - 4s 3ms/step - loss: 5832.4321 - accuracy: 0.4897
Epoch 2/175
804/804 [==============================] - 3s 3ms/step - loss: 34708.7578 - accuracy: 0.4969
Epoch 3/175
804/804 [==============================] - 3s 4ms/step - loss: 7487.0718 - accuracy: 0.4883
Epoch 4/175
804/804 [==============================] - 2s 3ms/step - loss: 18764.9805 - accuracy: 0.5141
Epoch 5/175
804/804 [==============================] - 1s 1ms/step - loss: 151.9041 - accuracy: 0.5310
Epoch 6/175
804/804 [==============================] - 1s 1ms/step - loss: 0.6944 - accuracy: 0.5310
Epoch 7/175
804/804 [==============================] - 1s 1ms/step - loss: 0.6915 - accuracy: 0.5310
Epoch 8/175
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5310
Epoch 9/175
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5310
Epoch 10/175
804/804 [==============================] - 2s 3ms/step - loss:

Optimization Attempt: 2

In [19]:
# Drop the non-beneficial columns
clean_application_df = application_df.drop(columns=["EIN", "NAME","USE_CASE", "SPECIAL_CONSIDERATIONS", "STATUS"])
clean_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,0,5000,1
1,T3,Independent,C2000,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,Association,0,5000,0
3,T3,CompanySponsored,C2000,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Trust,100000-499999,142590,1


In [20]:
# Determine the number of unique values in each column.
clean_application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [21]:
# Look at APPLICATION_TYPE value counts for binning
appCounts = clean_application_df["APPLICATION_TYPE"].value_counts()
appCounts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(appCounts[appCounts<500].index)
# Replace in dataframe
for app in application_types_to_replace:
   clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
clean_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [23]:
# Look at CLASSIFICATION value counts for binning
classCounts = clean_application_df["CLASSIFICATION"].value_counts()
classCounts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [24]:
# You may find it helpful to look at CLASSIFICATION value counts >10
classCounts10 = classCounts[classCounts>10]
classCounts10

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classCounts10[classCounts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
clean_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2143
C3000     1918
C2100     1883
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1256        2
C1246        2
C1267        2
C2190        1
C4200        1
C1370        1
C5200        1
C1820        1
C2600        1
C1900        1
C1248        1
C6100        1
C1236        1
C1732        1
C2570        1
C1580        1
C3700        1
C2500        1
C1570        1
C1283        1
C2380        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: CLASSIFICATION, dtype: int64

In [26]:
new_application_df= pd.get_dummies(clean_application_df)
new_application_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,5000,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,5000,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [27]:
# Split the preprocessed data into our features and target arrays (X,y)
y= new_application_df["IS_SUCCESSFUL"]
X= new_application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing datatset
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=n_input_features,units=70,activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10,activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 70)                5600      
                                                                 
 dense_5 (Dense)             (None, 10)                710       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 6,321
Trainable params: 6,321
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [31]:
# Train the model
fit_model= nn.fit(X_train,y_train,epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 48369.8867 - accuracy: 0.4929
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 31801.6582 - accuracy: 0.5020
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 42092.0781 - accuracy: 0.4897
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 25587.9824 - accuracy: 0.5019
Epoch 5/150
804/804 [==============================] - 1s 1ms/step - loss: 18451.5859 - accuracy: 0.5182
Epoch 6/150
804/804 [==============================] - 1s 1ms/step - loss: 10251.4883 - accuracy: 0.4907
Epoch 7/150
804/804 [==============================] - 1s 1ms/step - loss: 7897.1626 - accuracy: 0.5307
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 18225.5430 - accuracy: 0.5280
Epoch 9/150
804/804 [==============================] - 1s 1ms/step - loss: 7094.0503 - accuracy: 0.5313
Epoch 10/150
804/804 [==============================] - 1

Optimization Attempt: 3

In [ ]:
# Drop the non-beneficial columns
clean_application_df = application_df.drop(columns=["EIN","USE_CASE", "STATUS"])
clean_application_df.head()

In [ ]:
# Determine the number of unique values in each column.
clean_application_df.nunique()

In [ ]:
# Look at NAME value counts for binning
nameCounts= clean_application_df['NAME'].value_counts()
nameCounts

In [ ]:
# Choose a cutoff value and create a list of names to be replaced
# use the variable name `names_to_replace`
names_to_replace = list(nameCounts[nameCounts<300].index)
# Replace in dataframe
for app in application_types_to_replace:
   clean_application_df['NAME'] = clean_application_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
clean_application_df['NAME'].value_counts()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
appCounts = clean_application_df["APPLICATION_TYPE"].value_counts()
appCounts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(appCounts[appCounts<500].index)
# Replace in dataframe
for app in application_types_to_replace:
   clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
clean_application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classCounts = clean_application_df["CLASSIFICATION"].value_counts()
classCounts

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >10
classCounts10 = classCounts[classCounts>10]
classCounts10

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classCounts10[classCounts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
clean_application_df['CLASSIFICATION'].value_counts()

In [ ]:
new_application_df= pd.get_dummies(clean_application_df)
new_application_df

In [ ]:
# Split the preprocessed data into our features and target arrays (X,y)
y= new_application_df["IS_SUCCESSFUL"]
X= new_application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing datatset
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [43]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=n_input_features,units=70,activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=20,activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=5,activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 70)                1375500   
                                                                 
 dense_8 (Dense)             (None, 20)                1420      
                                                                 
 dense_9 (Dense)             (None, 5)                 105       
                                                                 
 dense_10 (Dense)            (None, 1)                 6         
                                                                 
Total params: 1,377,031
Trainable params: 1,377,031
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
# Train the model
fit_model= nn.fit(X_train,y_train,epochs=100)